# 04_LLM_Recommender.ipynb
🤖 Personalized Health Recommendations Using LLM Logic and Risk Drivers
## Introduction
This notebook bridges the gap between model explainability and actionable feedback by generating personalized health recommendations based on each individual's top risk drivers and predicted health risk score.

We use:
- The top 3 LIME-extracted drivers per individual (from the previous notebook)
- Their Health Risk Score (0–100)
- A logic-driven recommender function that simulates the behavior of an LLM by crafting empathetic, non-judgmental advice based on each feature

The recommendations aim to:
- Encourage self-awareness and gentle behavior change
- Be context-aware based on socio-demographic signals
- Reflect a supportive, human-centered tone to foster engagement

The final output is a structured CSV containing risk drivers, health scores, and the generated advice — suitable for integration into dashboards, portals, or counseling tools.

In [6]:
# 04_LLM_Recommender.ipynb
# Personalized Health Recommendations Using LLMs (based on risk drivers)

import pandas as pd
import time
import os

In [7]:
# ---------------------- # 1. Load LIME Top Drivers
lime_df = pd.read_csv("outputs/lime_top3_drivers_batch.csv")
scores_df = pd.read_csv("data/health_scores_scored_testset.csv")

merged = lime_df.merge(scores_df[['Health_Risk_Score']], left_on='Index', right_index=True)
print("Merged shape:", merged.shape)


Merged shape: (100, 5)


In [11]:
def recommend(drivers, risk_score):
    advice = []

    drivers = [str(d).lower() for d in drivers if pd.notna(d)]

    if any("bmi" in d for d in drivers):
        advice.append("Your body does so much for you every day. Let’s gently care for it through nourishing meals and joyful movement, at your own pace. Your body deserves kindness and patience.")

    if any("totinda" in d or "inactive" in d for d in drivers):
        advice.append("Even a short 10 mins walk or stretching session can uplift your mind and body. Movement doesn’t have to be intense to be meaningful.")

    if any("diabete" in d for d in drivers):
        advice.append("Blood sugar management is a journey. Your health journey is valid. Consider small, steady changes—more fiber, less sugar, and check-ins with your doctor when you can.")

    if any("income2" in d or "income" in d for d in drivers):
        advice.append("You’re doing your best in your circumstances. Your health matters, no matter your budget. Explore free or affordable community health options—they’re made for support like this.")

    if any("educa" in d or "education" in d for d in drivers):
        advice.append("Curiosity is a strength. A little learning—through videos, articles, or local health talks—can open big doors for wellbeing.")

    if any("smoker" in d for d in drivers):
        advice.append("There’s no shame in where you are. If you’re considering cutting back on smoking, know that every small step is a win.")

    if any("genhlth" in d or "general health" in d for d in drivers):
        advice.append("Your wellbeing matters deeply. Make space for rest, hydration, and little routines that bring you peace.")

    if any("employ1" in d or "employment" in d for d in drivers):
        advice.append("Work stress can be heavy. If you feel stretched, take even a few minutes each day just for you—your energy deserves recharging.")

    if any("age" in d and "<=" in d for d in drivers):
        advice.append("Starting young is a gift to your future self. These healthy habits you build now will carry you far.")

    # Add-on based on Risk Score
    if risk_score > 80:
        advice.append("It’s okay to be concerned. Your score suggests higher risk—this is a gentle nudge to consider a health check-up soon. You deserve care.")

    elif risk_score < 25:
        advice.append("You're doing beautifully. Keep honoring your body and mind with the care they deserve.")

    if not advice:
        advice.append("You’re on the right path. Be gentle with yourself, and keep walking at your pace—you’re doing better than you think.")

    return " ".join(advice)


In [12]:
# ---------------------- # 3. Generate Recommendations
recommendations = []

for _, row in merged.iterrows():
    drivers = [row['Top_1'], row['Top_2'], row['Top_3']]
    risk_score = row['Health_Risk_Score']
    rec = recommend(drivers, risk_score)

    recommendations.append({
        "Index": row['Index'],
        "Top_1": row['Top_1'],
        "Top_2": row['Top_2'],
        "Top_3": row['Top_3'],
        "Health_Risk_Score": round(risk_score, 2),
        "Recommendation": rec
    })

rec_df = pd.DataFrame(recommendations)
os.makedirs("outputs", exist_ok=True)
rec_df.to_csv("outputs/personalized_health_recommendations.csv", index=False)
print("✅ Saved to outputs/personalized_health_recommendations.csv")

✅ Saved to outputs/personalized_health_recommendations.csv


## Conclusion: Empathy-Driven, Feature-Aware Recommendations
This final step of the pipeline translates analytical outputs into human-friendly guidance using personalized logic chains inspired by LLM-style reasoning. Key outcomes:

- Crafted nuanced, condition-aware health tips tailored to risk factors like inactivity, education, general health, employment, and income
- Incorporated health risk severity using the score to modulate tone — from reassurance to stronger nudges for care
- Exported a CSV with 100 personalized recommendations for easy downstream use in web apps, reports, or health portals

This recommendation module adds interpretability, empathy, and actionability to the health risk pipeline — a critical step in turning data into meaningful change.